# Integrazione di Semantic Kernel con il server OpenBnB MCP

Questo notebook dimostra come utilizzare Semantic Kernel con il server OpenBnB MCP reale per cercare alloggi Airbnb autentici utilizzando MCPStdioPlugin. Per l'accesso a LLM, utilizza Azure AI Foundry. Per configurare le variabili di ambiente, puoi seguire la [Lezione di Configurazione](/00-course-setup/README.md)


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio
import subprocess
import sys


from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## Creazione della connessione al plugin MCP

Connetteremo al [server MCP OpenBnB](https://github.com/openbnb-org/mcp-server-airbnb) utilizzando MCPStdioPlugin. Questo server offre funzionalità di ricerca Airbnb tramite il pacchetto @openbnb/mcp-server-airbnb.


## Creazione del Client

In questo esempio, utilizzeremo Azure AI Foundry per accedere al nostro LLM. Assicurati che le variabili di ambiente siano configurate correttamente.


## Configurazione dell'Ambiente

Configura le impostazioni di Azure OpenAI. Assicurati di aver impostato le seguenti variabili d'ambiente:
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## Comprendere l'integrazione OpenBnB MCP

Questo notebook si connette al **vero server OpenBnB MCP** che fornisce funzionalità di ricerca Airbnb reali.

### Come funziona:

1. **MCPStdioPlugin**: Utilizza la comunicazione tramite input/output standard con il server MCP
2. **Pacchetto NPM reale**: Scarica ed esegue `@openbnb/mcp-server-airbnb` tramite npx
3. **Dati in tempo reale**: Restituisce dati reali sulle proprietà Airbnb dalle loro API
4. **Scoperta delle funzioni**: L'agente scopre automaticamente le funzioni disponibili dal server MCP

### Funzioni disponibili:

Il server OpenBnB MCP generalmente fornisce:
- **search_listings** - Cerca proprietà Airbnb per località e criteri
- **get_listing_details** - Ottieni informazioni dettagliate su proprietà specifiche
- **check_availability** - Verifica la disponibilità per date specifiche
- **get_reviews** - Recupera recensioni per le proprietà
- **get_host_info** - Ottieni informazioni sui proprietari delle proprietà

### Prerequisiti:

- **Node.js** installato sul tuo sistema
- **Connessione Internet** per scaricare il pacchetto del server MCP
- **NPX** disponibile (incluso con Node.js)

### Testare la connessione:

Puoi testare manualmente il server MCP eseguendo:
```bash
npx -y @openbnb/mcp-server-airbnb
```

Questo scaricherà e avvierà il server OpenBnB MCP, al quale Semantic Kernel si connetterà per ottenere dati reali da Airbnb.


## Esecuzione dell'Agent con il server OpenBnB MCP

Ora eseguiremo l'AI Agent che si collega al server OpenBnB MCP per cercare alloggi reali su Airbnb a Stoccolma per 2 adulti e 1 bambino. Sentiti libero di modificare la lista `user_inputs` per cambiare i criteri di ricerca.


In [ ]:
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        print("🚀 Starting with Azure OpenAI...")
        
        # Verify environment variables
        print("🔍 Checking Azure environment variables...")
        required_vars = ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME", "AZURE_OPENAI_ENDPOINT", "AZURE_OPENAI_API_KEY"]
        for var in required_vars:
            if os.getenv(var):
                print(f"✅ {var} is set")
            else:
                print(f"❌ {var} is NOT set")
        
        print("\n🔧 Creating MCP Plugin...")
        
        # Create MCP plugin connection to real OpenBnB server
        # Based on the GitHub repo, the server doesn't need special env vars
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb"],
        ) as airbnb_plugin:

            print("✅ MCP Plugin created and connected")
            
            # Wait a moment for the server to fully initialize
            await asyncio.sleep(2)
            
            # Try to list available tools
            try:
                tools = await airbnb_plugin.get_tools()
                print(f"🔧 Available tools: {[tool.name for tool in tools]}")
            except Exception as e:
                print(f"⚠️ Could not list tools: {str(e)}")

            # Create the Azure OpenAI service with proper configuration
            print("\n🤖 Creating Azure OpenAI service...")
            service = AzureChatCompletion(
                deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
                endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
                api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            )
            
            # Create agent with the service instance
            agent = ChatCompletionAgent(
                service=service,
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the available functions to search for properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("✅ Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 User: {user_input}")
                
                try:
                    # Use the simpler get_response method
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    
                    # Process the response text
                    response_text = str(response)
                    
                    # Remove any markdown code blocks around HTML
                    response_text = response_text.replace('```html', '').replace('```', '')
                    
                    # Display the result
                    print(f"🤖 {response.name}: {response_text[:200]}..." if len(response_text) > 200 else response_text)
                    
                    # If response contains HTML table, display it properly
                    if '<table' in response_text.lower():
                        # Add CSS styling for better table rendering
                        table_css = """
                        <style>
                            .airbnb-results table {
                                border-collapse: collapse;
                                width: 100%;
                                margin: 10px 0;
                            }
                            .airbnb-results th, .airbnb-results td {
                                border: 1px solid #ddd;
                                padding: 8px;
                                text-align: left;
                            }
                            .airbnb-results th {
                                background-color: #f2f2f2;
                                font-weight: bold;
                            }
                            .airbnb-results tr:nth-child(even) {
                                background-color: #f9f9f9;
                            }
                            .airbnb-results a {
                                color: #1976d2;
                                text-decoration: none;
                            }
                            .airbnb-results a:hover {
                                text-decoration: underline;
                            }
                        </style>
                        """
                        html_output = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                        display(HTML(html_output))
                    else:
                        # Display as regular text if no table
                        display(HTML(f'<div class="airbnb-results">{response_text}</div>'))
                        
                except Exception as e:
                    print(f"❌ Error processing user input: {str(e)}")
                    import traceback
                    traceback.print_exc()
                
            # Cleanup
            if thread:
                await thread.delete()
                print("🧹 Thread cleaned up")
                
    except Exception as e:
        print(f"❌ Main error: {str(e)}")
        import traceback
        traceback.print_exc()

# Run the main function
print("🚀 Starting MCP Agent...")
await main()
print("✅ Done!")

# Sommario
Congratulazioni! Hai costruito con successo un agente AI che si integra con la ricerca di alloggi reali utilizzando il Model Context Protocol (MCP):

## Tecnologie Utilizzate:
- Semantic Kernel - Per creare agenti intelligenti con Azure OpenAI
- Azure AI Foundry - Per funzionalità LLM e completamento delle chat
- MCP (Model Context Protocol) - Per l'integrazione standardizzata degli strumenti
- OpenBnB MCP Server - Per la funzionalità reale di ricerca su Airbnb
- Node.js/NPX - Per eseguire il server MCP esterno

## Cosa Hai Imparato:
- Integrazione MCP: Collegare agenti Semantic Kernel a server MCP esterni
- Accesso ai Dati in Tempo Reale: Ricerca di proprietà Airbnb reali tramite API live
- Comunicazione del Protocollo: Utilizzo della comunicazione stdio tra agente e server MCP
- Scoperta delle Funzioni: Scoprire automaticamente le funzioni disponibili dai server MCP
- Risposte in Streaming: Catturare e registrare le chiamate alle funzioni in tempo reale
- Rendering HTML: Formattare le risposte dell'agente con tabelle stilizzate e display interattivi

## Prossimi Passi:
- Integrare ulteriori server MCP (meteo, voli, ristoranti)
- Costruire un sistema multi-agente combinando i protocolli MCP e A2A
- Creare server MCP personalizzati per le tue fonti di dati
- Implementare una memoria conversazionale persistente tra le sessioni
- Distribuire l'agente su Azure Functions con orchestrazione del server MCP
- Aggiungere funzionalità di autenticazione utente e prenotazione



---

**Disclaimer**:  
Questo documento è stato tradotto utilizzando il servizio di traduzione automatica [Co-op Translator](https://github.com/Azure/co-op-translator). Sebbene ci impegniamo per garantire l'accuratezza, si prega di notare che le traduzioni automatiche possono contenere errori o imprecisioni. Il documento originale nella sua lingua nativa dovrebbe essere considerato la fonte autorevole. Per informazioni critiche, si raccomanda una traduzione professionale eseguita da un traduttore umano. Non siamo responsabili per eventuali fraintendimenti o interpretazioni errate derivanti dall'uso di questa traduzione.
